#  <font color=red> Module_25_程序與執行緒</font> 

# P25-3 subprocess
<ul>
    <li>
        <details>
            <summary style="font-siz: 1.5em;"><code>ping 8.8.8.8</code> in windows</summary>
            <img src="./img/ping_windows.png">
        </details>
    </li>
    <li>
        <details>
            <summary style="font-siz: 1.5em;"><code>ping 8.8.8.8</code> in unix</summary>
            <img src="./img/ping_ubuntu.png">
        </details>
    </li>
</ul>

In [ ]:
from subprocess import PIPE, Popen

command = "ping 8.8.8.8"  # unix: ping -c 5 8.8.8.8
with Popen(command, stdout=PIPE, shell=True) as process:
    output = process.communicate()[0].decode("cp950")  # window using cp950, unix using utf-8
    print(output)

---
# P25-5 threading

In [ ]:
import time

def heavy_io(task_id):
    """
    An simulated I/O intensive calculation, we simulate it with sleep.
    """
    print(f"task_{task_id} is start")
    time.sleep(2)
    print(f"task_{task_id} is done")

def sequential_do(n):
    for i in range(n):    
        heavy_io(i)

start = time.time()
sequential_do(10)
end = time.time()
print("Took: ", end - start)  # Took around 20 seconds in my computer

In [ ]:
import threading
import time

def heavy_io(task_id):
    """
    An simulated I/O intensive calculation, we simulate it with sleep.
    """
    print(f"task_{task_id} is start")
    time.sleep(2)
    print(f"task_{task_id} is done")

def threaded(n):
    threads = []
    for i in range(n):
        t = threading.Thread(target=heavy_io, args=(i,))
        threads.append(t)            # 將執行緒加入到列表中
        t.start()                    # 開始各個執行緒
    for t in threads:
        t.join()                     # 等待各個執行緒結束

start = time.time()
threaded(10)
end = time.time()
print("Took: ", end - start)  # Took around 2 seconds in my computer

---
# &spades; 補充

### threading 並不適用於CPU_bound的任務

In [ ]:
import time

def heavy_cpu(task_id):
    """
    A CPU heavy calculation, just as an example. This can be anything you like
    """
    print(f"task_{task_id} is start")
    n =800
    for x in range(1, n):
        for y in range(1, n):
            x**y
    print(f"task_{task_id} is done")

def sequential_do(n):
    for i in range(n):    
        heavy_cpu(i)


start = time.time()
sequential_do(10)
end = time.time()
print("Took: ", end - start)  # This takes about 25s on my system

In [ ]:
import threading
import time

def heavy_cpu(task_id):
    """
    A CPU heavy calculation, just as an example. This can be anything you like
    """
    print(f"task_{task_id} is start")
    n =800
    for x in range(1, n):
        for y in range(1, n):
            x**y
    print(f"task_{task_id} is done")


def threaded(n):
    threads = []
    for i in range(n):
        t = threading.Thread(target=heavy_cpu, args=(i,))
        threads.append(t)            # 將執行緒加入到列表中
        t.start()                    # 開始各個執行緒
    for t in threads:
        t.join()                     # 等待各個執行緒結束


start = time.time()
threaded(10)
end = time.time()
print("Took: ", end - start)  # This takes about 25s on my system
# The reason this is *not* an optimization for CPU bound functions, is the GIL!
# If the heavy function had a lot of blocking IO, like network calls or filesystem operations, this would  be a big optimization though

### 如果是CPU-bound任務，改用multiprocessing才能加快運算速度
- #### multiprocessing是透過subprocessing來實現的

In [ ]:
import time

def heavy_cpu(task_id):
    """
    A CPU heavy calculation, just as an example. This can be anything you like
    """
    print(f"task_{task_id} is start")
    n =800
    for x in range(1, n):
        for y in range(1, n):
            x**y
    print(f"task_{task_id} is done")

def sequential_do(n):
    for i in range(n):    
        heavy_cpu(i)


start = time.time()
sequential_do(10)
end = time.time()
print("Took: ", end - start)  # This takes about 25s on my system

In [ ]:
import time
import multiprocessing

def heavy_cpu(task_id):
    """
    A CPU heavy calculation, just as an example. This can be anything you like
    """
    print(f"task_{task_id} is start")
    n =800
    for x in range(1, n):
        for y in range(1, n):
            x**y
    print(f"task_{task_id} is done")


def multiproc(n):
    processes = []
    for i in range(n):
        p = multiprocessing.Process(target=heavy_cpu, args=(i,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()


start = time.time()
multiproc(10)
end = time.time()
print("Took: ", end - start)  # This takes about 7s on my system